# Traficom PC-Axis POC

In [136]:
import pandas as pd
import numpy as np
from pyaxis import pyaxis

In [21]:
def multiIndex_pivot(df, index = None, columns = None, values = None):
    # https://github.com/pandas-dev/pandas/issues/23955
    output_df = df.copy(deep = True)
    if index is None:
        names = list(output_df.index.names)
        output_df = output_df.reset_index()
    else:
        names = index
    output_df = output_df.assign(tuples_index = [tuple(i) for i in output_df[names].values])
    if isinstance(columns, list):
        output_df = output_df.assign(tuples_columns = [tuple(i) for i in output_df[columns].values])  # hashable
        output_df = output_df.pivot(index = 'tuples_index', columns = 'tuples_columns', values = values) 
        output_df.columns = pd.MultiIndex.from_tuples(output_df.columns, names = columns)  # reduced
    else:
        output_df = output_df.pivot(index = 'tuples_index', columns = columns, values = values)    
    output_df.index = pd.MultiIndex.from_tuples(output_df.index, names = names)
    return output_df


In [152]:

px = pyaxis.parse('/home/mikael/Downloads/katsastus_3.px', encoding='ISO-8859-2')
df = px['DATA']

df['Katsastusvuosi'] = df['Katsastusvuosi'].astype('int16')
df = df[~df['Käyttöönottovuosi'].str.contains('yhteens')]
df['Käyttöönottovuosi'] = df['Käyttöönottovuosi'].astype('int16', errors='ignore')
df = df[~df['Merkki ja mallisarja'].str.contains('yhteens')]
#df = df.infer_objects()
#df['DATA'] = df['DATA'].astype('Int64')
df['DATA'] = pd.to_numeric(df['DATA'])

print(df.sample(5))

df = df.pipe(multiIndex_pivot,
    index = ['Katsastusvuosi', 'Käyttöönottovuosi', 'Merkki ja mallisarja'],
    columns = ['Tiedot'],
    values = 'DATA')

df.to_pickle('/home/mikael/Downloads/katsastus_3.pickle.bz2', compression='bz2')


Katsastusvuosi Merkki ja mallisarja  Käyttöönottovuosi  \
7708              2017           Citroen C8               2005   
110097            2019              Mazda 2               2013   
103854            2019       Hyundai SONATA               2013   
4438              2017     Chevrolet CAMARO               2004   
21946             2017           Mazda CX-3               2010   

                        Tiedot  DATA  
7708                Hyväksytyt   NaN  
110097               Hylkäys-%   NaN  
103854  Katsastusten lukumäärä   NaN  
4438                Hyväksytyt   NaN  
21946               Hyväksytyt   NaN  


In [153]:
df = pd.read_pickle('/home/mikael/Downloads/katsastus_3.pickle.bz2')
#df = df.apply(pd.to_numeric)
#df.dtypes
df

Tiedot                                                Ajettujen kilometrien keskiarvo  \
Katsastusvuosi Käyttöönottovuosi Merkki ja mallisarja                                   
2017           2002              Alfa Romeo 147                              180000.0   
                                 Alfa Romeo 156                              209000.0   
                                 Alfa Romeo 159                                   NaN   
                                 Alfa Romeo GIULIETTA                             NaN   
                                 Alfa Romeo GT                                    NaN   
...                                                                               ...   
2019           2016              Volvo V70                                   121919.0   
                                 Volvo V90                                    93571.0   
                                 Volvo XC60                                   77012.0   
                                 Volvo XC70                                   96777.0   
                                 Volvo XC90                                   71800.0   

Tiedot                                                Ajettujen kilometrien mediaani  \
Katsastusvuosi Käyttöönottovuosi Merkki ja mallisarja                                  
2017           2002              Alfa Romeo 147                             179000.0   
                                 Alfa Romeo 156                             214000.0   
                                 Alfa Romeo 159                                  NaN   
                                 Alfa Romeo GIULIETTA                            NaN   
                                 Alfa Romeo GT                                   NaN   
...                                                                              ...   
2019           2016              Volvo V70                                   90125.0   
                                 Volvo V90                                   79184.0   
                                 Volvo XC60                                  70641.0   
                                 Volvo XC70                                  81461.0   
                                 Volvo XC90                                  68949.0   

Tiedot                                                Hylkäys-% Hylätyt  \
Katsastusvuosi Käyttöönottovuosi Merkki ja mallisarja                     
2017           2002              Alfa Romeo 147            54.0    54.0   
                                 Alfa Romeo 156            43.0    55.0   
                                 Alfa Romeo 159             NaN     NaN   
                                 Alfa Romeo GIULIETTA       NaN     NaN   
                                 Alfa Romeo GT              NaN     NaN   
...                                                         ...     ...   
2019           2016              Volvo V70                  7.0    27.0   
                                 Volvo V90                  3.0    11.0   
                                 Volvo XC60                 1.0    11.0   
                                 Volvo XC70                 1.0     4.0   
                                 Volvo XC90                 2.0     7.0   

Tiedot                                                Hyväksytyt  \
Katsastusvuosi Käyttöönottovuosi Merkki ja mallisarja              
2017           2002              Alfa Romeo 147             47.0   
                                 Alfa Romeo 156             72.0   
                                 Alfa Romeo 159              NaN   
                                 Alfa Romeo GIULIETTA        NaN   
                                 Alfa Romeo GT               NaN   
...                                                          ...   
2019           2016              Volvo V70                 363.0   
                                 Volvo V90                 422.0   
                                 Volvo XC

In [140]:
#df.loc[(['2017'], ['2002'], ['Alfa Romeo 147'])]
df.index
#ind = df['Merkki ja mallisarja']
#ind
#df[~df['Merkki ja mallisarja'].str.contains('yhteens')]

MultiIndex([(2017, 2002,       'Alfa Romeo 147'),
            (2017, 2002,       'Alfa Romeo 156'),
            (2017, 2002,       'Alfa Romeo 159'),
            (2017, 2002, 'Alfa Romeo GIULIETTA'),
            (2017, 2002,        'Alfa Romeo GT'),
            (2017, 2002,      'Alfa Romeo MITO'),
            (2017, 2002,              'Audi A1'),
            (2017, 2002,              'Audi A2'),
            (2017, 2002,              'Audi A3'),
            (2017, 2002,              'Audi A4'),
            ...
            (2019, 2016,            'Volvo S80'),
            (2019, 2016,            'Volvo S90'),
            (2019, 2016,            'Volvo V40'),
            (2019, 2016,            'Volvo V50'),
            (2019, 2016,            'Volvo V60'),
            (2019, 2016,            'Volvo V70'),
            (2019, 2016,            'Volvo V90'),
            (2019, 2016,           'Volvo XC60'),
            (2019, 2016,           'Volvo XC70'),
            (2019, 2016,          